In [ ]:
from meteostat import Stations, Daily
from datetime import datetime
import pandas as pd

start = datetime(2018, 1, 1)
end = datetime(2018, 12, 31)

# Coordinates of several major Slovenian cities
locations = {
    'Ljubljana': (46.05, 14.51),
    'Maribor': (46.56, 15.64),
    'Koper': (45.55, 13.73),
    'Celje': (46.23, 15.27)
}

dfs = []

for city, (lat, lon) in locations.items():
    station = Stations().nearby(lat, lon).fetch(1)
    
    if not station.empty:
        station_id = station.index[0]  # The station ID is now in the index
        data = Daily(station_id, start, end).fetch()
        data = data.rename(columns=lambda x: f"{city}_{x}")
        dfs.append(data)
    else:
        print(f"⚠️ No station found near {city}.")

# Combine and compute mean across all cities
if dfs:
    combined = pd.concat(dfs, axis=1)
    aggregated = combined.groupby(level=0, axis=1).mean()
    aggregated.to_csv("slovenia_weather_aggregated.csv")
    print("✅ Saved: slovenia_weather_aggregated.csv")
else:
    print("❌ No dataframes to combine.")


In [ ]:
import pandas as pd

# Load the CSV
df = pd.read_csv("slovenia_weather_aggregated.csv", parse_dates=["time"])
df = df.set_index("time")

# Extract unique feature types from the column names
features = ['prcp', 'pres', 'snow', 'tavg', 'tmax', 'tmin', 'tsun', 'wdir', 'wpgt', 'wspd']

# Initialize dictionary to hold averaged columns
avg_data = {}

for feature in features:
    # Get all columns that end with the current feature
    feature_cols = [col for col in df.columns if col.endswith(f"_{feature}")]
    
    if feature_cols:
        # Average while skipping NaNs
        avg_data[feature] = df[feature_cols].mean(axis=1)

# Create DataFrame from the averaged values
df_avg = pd.DataFrame(avg_data, index=df.index)

# Optional: Save to CSV
df_avg.to_csv("slovenia_weather_averaged.csv")
